In [1]:
import xarray as xr
import stmtools

stmat = xr.open_zarr('../data/stm.zarr')
stmat

<xarray.Dataset>
Dimensions:    (points: 318225, time: 10)
Coordinates:
    azimuth    (points) int64 dask.array<chunksize=(5000,), meta=np.ndarray>
    lat        (points) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    lon        (points) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    range      (points) int64 dask.array<chunksize=(5000,), meta=np.ndarray>
  * time       (time) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: points
Data variables:
    amplitude  (points, time) float32 dask.array<chunksize=(5000, 10), meta=np.ndarray>
    complex    (points, time) complex64 dask.array<chunksize=(5000, 10), meta=np.ndarray>
    phase      (points, time) float32 dask.array<chunksize=(5000, 10), meta=np.ndarray>

In [2]:
subset = stmat.stm.subset(method='threshold', var='amplitude', threshold='>1')
subset

<xarray.Dataset>
Dimensions:    (points: 316762, time: 10)
Coordinates:
    azimuth    (points) int64 dask.array<chunksize=(5000,), meta=np.ndarray>
    lat        (points) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    lon        (points) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    range      (points) int64 dask.array<chunksize=(5000,), meta=np.ndarray>
  * time       (time) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: points
Data variables:
    amplitude  (points, time) float32 dask.array<chunksize=(5000, 10), meta=np.ndarray>
    complex    (points, time) complex64 dask.array<chunksize=(5000, 10), meta=np.ndarray>
    phase      (points, time) float32 dask.array<chunksize=(5000, 10), meta=np.ndarray>

In [14]:
#pipeline = pdal.Filter.mortonorder().pipeline(subset)
#pipeline = pdal.Filter.sort(dimension="lat").pipeline(subset.to_array())
#print(pipeline.execute())

In [1]:
import pdal

data = "/storage/nD_PointClouds/data/entwine/red-rocks-mpi.laz"

pipeline_read = pdal.Reader.las(filename=data).pipeline()
print(pipeline_read.execute())  # 1065 points

4004326


In [2]:
# Get the data from the first array
# [array([(637012.24, 849028.31, 431.66, 143, 1,
# 1, 1, 0, 1,  -9., 132, 7326, 245380.78254963,  68,  77,  88),
# dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'),
# ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'),
# ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'),
# ('UserData', 'u1'), ('PointSourceId', '<u2'),
# ('GpsTime', '<f8'), ('Red', '<u2'), ('Green', '<u2'), ('Blue', '<u2')])
arr = pipeline_read.arrays[0]

In [3]:
# Filter out entries that have Z > 1940
z = arr[arr["Z"] > 1960]
print(len(z))  # 704 points

674215


In [4]:
# Now use pdal to clamp points that have z 1970 <= v < 1980
pipeline_clamp = pdal.Filter.range(limits="Z[1970:1980)").pipeline(z)
print(pipeline_clamp.execute())  # 387 points
clamped = pipeline_clamp.arrays[0]

153927


In [5]:
pipeline_sort = pdal.Filter.sort(dimension="X").pipeline(arr)
print(pipeline_sort.execute())

4004326


In [11]:
pipeline_morton = pdal.Filter.mortonorder().pipeline(arr)
print(pipeline_morton.execute())

4004326


In [14]:
import numpy as np

arr_morton = pipeline_morton.arrays[0]
arr_morton["Intensity"] = np.array(range(0, len(arr_morton))) % 65536
arr_morton["Intensity"]

array([   0,    1,    2, ..., 6627, 6628, 6629], dtype=uint16)

In [15]:
output_data = "/storage/nD_PointClouds/data/entwine/red-rocks-mpi_morton.laz"
pipeline = pdal.Writer.las(
    filename=output_data,
    offset_x="auto",
    offset_y="auto",
    offset_z="auto",
    scale_x=0.01,
    scale_y=0.01,
    scale_z=0.01,
).pipeline(arr_morton)
print(pipeline.execute())

4004326
